In [1]:
#Single task network to compare with Vlado


import torch
from bayes_opt import BayesianOptimization
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from math import sqrt
import random
import itertools

#Load some of the data
exp_data = pd.read_csv('../exp.tab', sep='\t', index_col=0)
cnv_data = pd.read_csv('../cnv.tab', sep='\t', index_col=0)
ydat = pd.read_csv('../labels.tab', sep='\t', index_col=0)
train_activity_data = pd.read_csv('../train_activity.tab', sep='\t')
test_activity_data = pd.read_csv('../test_activity.tab', sep ='\t')

#labels
traininglabels = train_activity_data.columns[1:]
testinglabels = test_activity_data.columns[1:]

#concatenate two data frames
frames = [exp_data, cnv_data]

xdatw = pd.concat(frames)
traininglabels

Index(['786O_KIDNEY', 'A1207_CENTRAL_NERVOUS_SYSTEM',
       'A172_CENTRAL_NERVOUS_SYSTEM', 'A204_SOFT_TISSUE', 'A2058_SKIN',
       'A549_LUNG', 'AGS_STOMACH', 'AML193_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE',
       'ASPC1_PANCREAS', 'BT20_BREAST',
       ...
       'TE9_OESOPHAGUS', 'THP1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE',
       'TOV112D_OVARY', 'TYKNU_OVARY', 'U178_CENTRAL_NERVOUS_SYSTEM',
       'U343_CENTRAL_NERVOUS_SYSTEM', 'U87MG_CENTRAL_NERVOUS_SYSTEM',
       'UOK101_KIDNEY', 'VCAP_PROSTATE', 'ZR7530_BREAST'],
      dtype='object', length=142)

In [27]:
#Deep Learning Net Class

class EssentialityNet:

    def __init__(self):
        self.inputnum = xdatw.shape[0]
        self.trainscores = []
        self.testscoreslist = []
        self.learning_rate = 0.00009
        self.H = 100
        self.n_iter = 300 #training iterations
        self.minimum = 100000
        self.stopcounter = 3
        self.layernum = 1
        self.layers = []
                
        #model
        self.model = torch.nn.Sequential(
        torch.nn.Linear(self.inputnum, self.H),
        torch.nn.ReLU(),
        torch.nn.Linear(self.H, 1),
        )
        
        #set loss function and optimizer
        self.loss = torch.nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
    
    #plot scores
    def plot(self, trainscores, testscores):
        x = np.arange(self.n_iter)
        plt.plot(x, self.trainscores, label='Train')
        plt.title('Training vs Test Accuracy')
        plt.xlabel('NN Training Iterations')
        plt.ylabel('Accuracy')
    
        plt.plot(np.asarray(x), np.asarray(testscores), label='Test') #plot
        plt.legend()
        
    #sets the proper method
    def setModel(self, Layernum, Neuronnum):  
        
        self.layernum = int(round(Layernum))
        self.H = int(round(Neuronnum))
        
        #initial input layer
        self.layers.append(torch.nn.Linear(self.inputnum, self.H))
        
        for n in range(self.layernum):
            if n != 0:
                self.layers.append(torch.nn.Linear(self.H, self.H))
            self.layers.append(torch.nn.ReLU())
            
        self.layers.append(torch.nn.Linear(self.H, 1))
        
        #set the method to whatever layers were chosen
        self.model = torch.nn.Sequential(*self.layers)
    
    def setRegularization(self, L2Reg):
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate, weight_decay= L2Reg)

    def fit(self, xtrain, ytrain, xtest, ytest):
      
        #convert to variables
        xtrain_var = Variable(torch.FloatTensor(xtrain))
        xtest_var = Variable(torch.FloatTensor(xtest))
        ytrain_var = Variable(torch.FloatTensor(ytrain))
        ytest_var = Variable(torch.FloatTensor(ytest))
        
        for t in range(self.n_iter):
        
            #calculate loss
            ypred = self.model(xtrain_var)

            diff = self.loss(ypred, ytrain_var)
            self.trainscores.append(diff.data[0])
            
            #test performance
            ypredtest = self.model(xtest_var)
            difftest = self.loss(ypredtest, ytest_var)
            
            #find the best point
            if t > 10 and self.minimum < difftest.data[0]:
                self.stopcounter -= 1

                if self.stopcounter == 0:
                    self.n_iter = t
                    self.trainscores.pop()
                    break
            elif t > 10 and self.stopcounter < 3:
                self.stopcounter += 1
            
            self.minimum = difftest.data[0]
            
            self.testscoreslist.append(difftest.data[0])
            
            #zero gradients
            self.optimizer.zero_grad()
            #backpropagate
            diff.backward() 
            #update weights
            self.optimizer.step() 

    # predict with the test data
    def predict(self, X):
        
        X_var = Variable(torch.FloatTensor(X))
        predicts = self.model(X_var)
        return predicts.data.numpy()
    
#other functions for running the nn

def figureoutnetwork(layernum, neuronnum, l2reg):
    n = EssentialityNet()
    n.setModel(layernum, neuronnum)
    n.setRegularization(l2reg)
            
    n.fit(xtrain_val, ytrain_val, xtest_val, ytest_val)
    predictions = n.predict(xtest)
    return(calculateRMSE_single(predictions, ytest))
#     saveRMSE(predictions, ytest)

def figureoutnetwork3(neuronnum, l2reg):
    n = EssentialityNet()
    n.setModel(3, neuronnum)
    n.setRegularization(l2reg)
            
    n.fit(xtrain_val, ytrain_val, xtest_val, ytest_val)
    predictions = n.predict(xtest)
    return(calculateRMSE(predictions, ytest))
 
#calculate RMSE function
def calculateRMSE_single(predicts, actuals):
#     mses = []  
#     multitaskrmses = []
#     preds = predicts.data.numpy()

#     for i in range(preds.shape[1]):
#         mses.append(((preds[:,i] - actuals[:,i])**2).mean())
#         multitaskrmses.append(sqrt(mses[i]))

#     print(len(multitaskrmses)) 
#     preds = predicts.data.numpy()

    
#     print(len(predicts))
    predicts2 = [val for sublist in predicts for val in sublist]
#     print(len(actuals))
    print(predicts2)
    print(actuals)

    return sqrt(((predicts2 - actuals)**2).mean())

#calculate RMSE function
def calculateRMSE(predicts, actuals):
    mses = []  
    multitaskrmses = []
    preds = predicts.data.numpy()

    for i in range(preds.shape[1]):
        mses.append(((preds[:,i] - actuals[:,i])**2).mean())
        multitaskrmses.append(sqrt(mses[i]))

    print(len(multitaskrmses))       
    return(np.mean(multitaskrmses))

def saveRMSE(predicts, actuals):
    mses = []  
    multitaskrmses = []
    preds = predicts.data.numpy()

    for i in range(preds.shape[1]):
        mses.append(((preds[:,i] - actuals[:,i])**2).mean())
        multitaskrmses.append(sqrt(mses[i]))
    
    #open a file for saving rmses
    rmses_file = open('rmses_single_' + str(fileno) + ".tab", 'w')
    
    for item in multitaskrmses:
          rmses_file.write("%s\n" % item)

In [30]:
#run for first 5 genes
mselist = {}
for i in range(5):
    
    #initialize
    n = EssentialityNet()
          
    #best tasks
    topTasks = pd.read_csv("../combined_stats.tab", sep='\t')
    tasks = topTasks.iloc[:,0].values
    
    goodydat = ydat.transpose()[tasks]
    goodydat = goodydat.transpose()
    
    #get name
    name = goodydat.index[i]
    
    #get training and testing labels that vlado used
    traininglabelsdat = pd.read_csv('../train_test_splits/train_test_splits/' + name + '_train_idx', header = None)
    testinglabelsdat = pd.read_csv('../train_test_splits/train_test_splits/' + name + '_test_idx', header = None)
    
    #convert them to lists
    traininglabels = traininglabelsdat.iloc[:,0].values
    testinglabels = testinglabelsdat.iloc[:,0].values

    #index the data 
    xtraindat = xdatw[traininglabels].transpose()
    xtestdat = xdatw[testinglabels].transpose()
    ytraindat = goodydat[traininglabels].transpose()
    ytestdat = goodydat[testinglabels].transpose()
    
    #normalize inputs
    xtrain = preprocessing.scale(xtraindat)
    xtest = preprocessing.scale(xtestdat)

    #get column (task)
    ytrain = np.array(ytraindat.iloc[:,i])  
    ytest = np.array(ytestdat.iloc[:,i]) 
    
#     print(xtrain)
#     print(ytrain)
#     print(xtrain_val)
    
    #split for validation
    xtrain_val, xtest_val, ytrain_val, ytest_val = train_test_split(xtrain, ytrain, test_size=0.2, random_state=434)
    
    print(i)
    print(name)

    #fit and predict
    print("RMSE: ")
    print(figureoutnetwork(3, 356, 0.012))
    #3 356
#     torch.save(n, "/home/user/Documents/stuart_research/data/data/exp_cnv_activity/models/" + name + ".txt")

0
AADAT_1_11010
RMSE: 
0.9149543118242022
1
AASDHPPT_1_10110
RMSE: 
1.076485023604067
2
ABCB1_1_11100
RMSE: 
1.000051445413451
3
ABCB5_1_01010
RMSE: 
1.213929665661624
4
ABCB7_1_10111
RMSE: 
1.2723682081495553
